In [1]:
# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow as tf

/Users/bp/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bp/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bp/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bp/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: Futu

# Get the data

In [2]:
housing = pd.read_csv("housing.csv")
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
#Drop the missing values
housing.dropna(axis=0, inplace=True)

# Let's also reset the index
housing.reset_index(inplace=True, drop=True)


housing.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


# Prepare the data for Machine Learning algorithms

In [4]:
#Set the training and test data sets
housing_num = housing.drop("ocean_proximity", axis=1) # drop labels 

#Select the label
housing_target = housing[["ocean_proximity"]]

### Standardize the data

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing_num_std = scaler.fit_transform(housing_num)

In [6]:
housing_num_std

array([[-1.32731375,  1.05171726,  0.98216331, ..., -0.97683327,
         2.34516291,  2.12881864],
       [-1.32232256,  1.04235526, -0.60621017, ...,  1.67037262,
         2.33263161,  1.31362603],
       [-1.33230494,  1.03767426,  1.85576873, ..., -0.84342665,
         1.78293943,  1.25818254],
       ...,
       [-0.82320322,  1.77727236, -0.92388486, ..., -0.17377773,
        -1.14317103, -0.99247676],
       [-0.87311515,  1.77727236, -0.84446619, ..., -0.39350628,
        -1.05513604, -1.05831591],
       [-0.83318561,  1.74918635, -1.00330353, ...,  0.07995643,
        -0.78060586, -1.01759959]])

In [7]:
housing_num_std.shape

(20433, 9)

### Create the label column

Tensorflow wants the labels in integer form. So, we need to do Ordinal Encoding, then convert the numbers to integers.

In [9]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

housing_labels_ord = ordinal_encoder.fit_transform(housing_target)

housing_labels_ord[:10]

array([[3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.],
       [3.]])

In [10]:
# Data type is float. It needs to be integer
housing_labels_ord.dtype

dtype('float64')

In [11]:
#Convert to integer

housing_labels_int = housing_labels_ord.astype(int)

housing_labels_int.dtype

dtype('int64')

In [52]:
housing_labels_int.shape

(20433, 1)

# Split data (train/test)

In [53]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(housing_num_std, housing_labels_int, test_size=0.3)

# Multiclass classification using Keras



In [54]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# fix random seed for reproducibility
np.random.seed(42)

In [55]:
#Define the model: for multi-class

model = Sequential()

model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))


In [56]:
# Compile model

#Optimizer:
sgd = keras.optimizers.SGD(lr=0.05)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [57]:
# Fit the model

model.fit(train_x, train_y, epochs=100, batch_size=100)

Epoch 1/100
14303/14303 [==============================] - 0s 18us/step - loss: 1.1889 - accuracy: 0.5263
Epoch 2/100
14303/14303 [==============================] - 0s 9us/step - loss: 0.8298 - accuracy: 0.7085
Epoch 3/100
14303/14303 [==============================] - 0s 9us/step - loss: 0.6589 - accuracy: 0.7686
Epoch 4/100
14303/14303 [==============================] - 0s 9us/step - loss: 0.5840 - accuracy: 0.7880
Epoch 5/100
14303/14303 [==============================] - 0s 9us/step - loss: 0.5296 - accuracy: 0.7948
Epoch 6/100
14303/14303 [==============================] - 0s 10us/step - loss: 0.4776 - accuracy: 0.8077
Epoch 7/100
14303/14303 [==============================] - 0s 12us/step - loss: 0.4245 - accuracy: 0.8303
Epoch 8/100
14303/14303 [==============================] - 0s 10us/step - loss: 0.5096 - accuracy: 0.8186
Epoch 9/100
14303/14303 [==============================] - 0s 10us/step - loss: 0.3834 - accuracy: 0.8460
Epoch 10/100
14303/14303 [========================

In [58]:
# evaluate the model

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


6130/6130 [==============================] - 0s 22us/step

accuracy: 84.96%


# Optimizers, Learning rate, Dropout, Initialization & Activation functions

In [59]:
#Define the model: for multi-class


#Set the learning rate:
lr=0.001


#Available optimizers:
adagrad = keras.optimizers.Adagrad(lr=lr, decay=0.0)
sgd = keras.optimizers.SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = keras.optimizers.RMSprop(lr=lr, rho=0.9, decay=0.0)
adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
nesterov_adam = keras.optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, schedule_decay=0.004)

#Initializations:
xavier = keras.initializers.glorot_normal(seed=None)
he = keras.initializers.he_normal(seed=None)


# Activation functions. Uncomment only one
#activation = 'elu' 
activation = 'relu'
#activation = 'tanh'
#activation = 'sigmoid'



#See the droput layers below:
model = Sequential()

model.add(Dense(50, input_dim=9, activation=activation, kernel_initializer=xavier))

model.add(Dropout(0.2))

model.add(Dense(25, activation=activation, kernel_initializer=xavier))

model.add(Dropout(0.2))

model.add(Dense(10, activation=activation, kernel_initializer=xavier))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))



#Compile"
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=nesterov_adam, metrics=['accuracy'])

In [60]:
# Fit the model
model.fit(train_x, train_y, epochs=100, batch_size=100)

Epoch 1/100
14303/14303 [==============================] - 0s 33us/step - loss: 1.2907 - accuracy: 0.5246
Epoch 2/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.8433 - accuracy: 0.6971
Epoch 3/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.7321 - accuracy: 0.7328
Epoch 4/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.6685 - accuracy: 0.7503
Epoch 5/100
14303/14303 [==============================] - 0s 17us/step - loss: 0.6188 - accuracy: 0.7621
Epoch 6/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.5817 - accuracy: 0.7728
Epoch 7/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.5385 - accuracy: 0.7851
Epoch 8/100
14303/14303 [==============================] - 0s 18us/step - loss: 0.5008 - accuracy: 0.7961
Epoch 9/100
14303/14303 [==============================] - 0s 16us/step - loss: 0.4800 - accuracy: 0.8046
Epoch 10/100
14303/14303 [====================

In [61]:

# evaluate the model
scores = model.evaluate(test_x, test_y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6130/6130 [==============================] - 0s 27us/step

accuracy: 93.67%


# Early stopping based on validation results

To do this, you need to send the validation data sets to the fit() function and use a callback.

EarlyStopping Arguments:

**monitor:** quantity to be monitored.<br>
**min_delta:** minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.<br>
**patience:** number of epochs with no improvement after which training will be stopped.<br>
**verbose:** verbosity mode.<br>
**mode:** one of {auto, min, max}. In min mode, training will stop when the quantity monitored has stopped decreasing; in max mode it will stop when the quantity monitored has stopped increasing; in auto mode, the direction is automatically inferred from the name of the monitored quantity.<br>
**baseline:** Baseline value for the monitored quantity to reach. Training will stop if the model doesn't show improvement over the baseline.<br>
**restore_best_weights:** whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.

In [62]:
from keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

model.fit(train_x, train_y, validation_data=(test_x, test_y), 
          epochs=100, batch_size=100, callbacks=callback)

Train on 14303 samples, validate on 6130 samples
Epoch 1/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.2004 - accuracy: 0.9169 - val_loss: 0.1527 - val_accuracy: 0.9396
Epoch 2/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.1973 - accuracy: 0.9199 - val_loss: 0.1496 - val_accuracy: 0.9393
Epoch 3/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.1945 - accuracy: 0.9209 - val_loss: 0.1467 - val_accuracy: 0.9405
Epoch 4/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.1952 - accuracy: 0.9210 - val_loss: 0.1510 - val_accuracy: 0.9418
Epoch 5/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.1956 - accuracy: 0.9199 - val_loss: 0.1515 - val_accuracy: 0.9414
Epoch 6/100
14303/14303 [==============================] - 0s 19us/step - loss: 0.1937 - accuracy: 0.9179 - val_loss: 0.1607 - val_accuracy: 0.9313
Epoch 7/100
14303/14303 [==============================] - 0s 1

In [63]:
scores = model.evaluate(test_x, test_y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

6130/6130 [==============================] - 0s 15us/step

accuracy: 93.52%
